## Field Level Encryption - Example using Symmetric Key Encryption

In [1]:
import pandas as pd
import requests
import io

from Crypto.Cipher import AES
from Crypto.Hash import MD5
from typing import Tuple

#### Functions to encrypt and decrypt

In [5]:
def aes_encrypt(value:bin, key:bin)-> Tuple[bin,bin]:
    """
    Function to perform AES encryption using SIV mode
    Args:
        value: plaintext value to be encrypted (aka. secret message)
        key: encryption key used to perform encryption (aka. secret key)
    
    Return:
        ciphertext and digest in binary data type
    """
    
    # convert key to MD5 as SIV mode requires a minimum 32 byte key
    hash_key = MD5.new(key).hexdigest().encode('ascii')
    
    # generates a new AES key
    cipher = AES.new(hash_key, AES.MODE_SIV)
    
    ciphertext, digest = cipher.encrypt_and_digest(value)
    
    return ciphertext, digest


def aes_decrypt(ciphertext:bin, digest:bin, key:bin) -> bin:
    """
    Function to perform AES decrhyption using SIV mode
    
    Args:
        ciphertext: ciphertext to be decrypted
        digest: digest produced from encryption
        key: encryption used to perform decryption
    
    Return:
        ciphertext and digest in binary type
    """
    
    # convert key to MD5 as SIV mode requires a minimun 32 byte key
    hash_key = MD5.new(key).hexdigest().encode('ascii')
    
    # generate a new AES key
    cipher = AES.new(hash_key, AES.MODE_SIV)
    
    return cipher.decrypt_and_verify(ciphertext, digest)


def decrypt_and_check_integrity(ciphertext_1, digest_1, secret_key):
    try:
        print('The secret message is: ', aes_decrypt(ciphertext_1, digest_1, secret_key))
    except ValueError:
        print('The message has been tampered with!')
        
    return

#### Perform encryption on a simple message

In [6]:
secret_key = b'a secret key'
message = b'a random secret message'

ciphertext_1, digest_1 = aes_encrypt(message, secret_key)

#### Try to decrypt the ciphertext

In [7]:
decrypt_and_check_integrity(ciphertext_1, digest_1, secret_key)

The secret message is:  b'a random secret message'


#### If the ciphertext has been tampered with

In [8]:
tampered_ciphertext = b'\x17V\xce\x1f\xd9\x9c\xbeE\xee$\xe7g\x811\x1f\xe5\xba\xf8\xf2\x11\x85\x1b\x86'
decrypt_and_check_integrity(tampered_ciphertext, digest_1, secret_key)

The message has been tampered with!


## Exercise

#### Get data from data.gov.sg

Note: to perform the download, you need to unrestrict the download rate limit for jupyter notebook by re-opening it with command:

``` 
jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10 
```

In [9]:
api_response = requests.get('https://data.gov.sg/api/action/package_show?id=acra-information-on-corporate-entities'
                           ).json()
resources = api_response['result']['resources']

data_set_url = ''

for resource in resources:
    if resource['name'] == 'ACRA Information on Corporate Entities (\'X\')':
        data_set_url = resource['url']
        print('Data set URL:\n{}'.format(data_set_url))
        
acra_csv_data = io.StringIO(requests.get(data_set_url).content.decode('utf-8'))

acra_csv_df = pd.read_csv(acra_csv_data, dtype=str)

acra_csv_df.head()

Data set URL:
https://storage.data.gov.sg/acra-information-on-corporate-entities/resources/acra-information-on-corporate-entities-x-2021-12-13T02-37-28Z.csv


,business_constitution_description,primary_ssic_description,primary_user_described_activity,street_name,entity_status_description,annual_return_date,postal_code,paid_up_capital10_preference,entity_name,paid_up_capital2_others,...,paid_up_capital10_ordinary,paid_up_capital10_others,uen_of_audit_firm4,paid_up_capital7_others,uen_of_audit_firm5,entity_type_description,paid_up_capital3_ordinary,paid_up_capital5_ordinary,paid_up_capital9_preference,paid_up_capital1_ordinary
0,Sole Proprietor,PUBS,na,CIRCULAR ROAD,Cancelled,na,049392,na,X,na,...,na,na,na,na,na,Business,na,na,na,na
1,Partnership,WHOLESALE TRADE OF A VARIETY OF GOODS WITHOUT ...,na,SENJA ROAD,na,na,670616,na,X & B TRADING,na,...,na,na,na,na,na,Business,na,na,na,na
2,na,CAFES AND COFFEE HOUSES,SNACKS AND BEVERAGE KIOSK OR SHOP,ROBINSON ROAD,Struck Off,2013-09-25T18:54:24,048545,na,X & E PASSION PTE. LTD.,na,...,na,na,na,na,na,Local Company,na,na,na,100000
3,Sole Proprietor,RENTAL AND LEASING OF CARS WITH DRIVER (EXCLUD...,na,CHOA CHU KANG CRESCENT,Cancelled (Non-Renewal),na,682691,na,X & F SERVICES,na,...,na,na,na,na,na,Business,na,na,na,na
4,na,OTHER HOLDING COMPANIES,INVESTMENT,CECIL STREET,Struck Off,na,049705,na,X & H INTERNATIONAL PTE. LTD.,na,...,na,na,na,na,na,Local Company,na,na,na,1


```
1. Perform encryption on field 'entity_name'
```

In [12]:
######################
####YOUR CODE HERE####
######################

encrypted_df['entity_name'].head()

0    (b'\xdf', b'KkA\xaf\xdb\x88\xf3\ni\xcc\xda\xf6...
1    (b'\x83O\x18\x82{\x814\xff\x0cB\x0f-\x14', b'\...
2    (b'\x82<5c\x80\x04\xc3\x92\x83\x06\x02\xd4\xf1...
3    (b'\n=\xec\xc8a\x01&3\xe3m\xa6hlP', b'\x1fE.\x...
4    (b'$\x84\xe0\x18\xdey\xd6.f\xc7\x12U\xfa|\xedh...
Name: entity_name, dtype: object

```
2. Perform decryption on field 'entity_name'
```

In [11]:
######################
####YOUR CODE HERE####
######################

0                                X
1                    X & B TRADING
2          X & E PASSION PTE. LTD.
3                   X & F SERVICES
4    X & H INTERNATIONAL PTE. LTD.
Name: entity_name, dtype: object

## End of Exercise